In [62]:
from matplotlib import pyplot as plt
%matplotlib notebook

import sys 
sys.path.append('..')

from ParserQuery import *
from Index import Index
from Weighter import WeighterVector, WeighterSchema3
from IRModel import Vectoriel
from Evaluation import *
from EvalIRModel import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
index = Index('easyCLEF08', '../easyCLEF08/easyCLEF08_text.txt', index_path='../indexes/easyCLEF08/')
#index.indexation()
weighter2 = WeighterVector(index)
#weighter2.calculeNorms()
weighter3 = WeighterSchema3(index)
#weighter3.calculeNorms()
irm2 = Vectoriel(weighter2)
irm3 = Vectoriel(weighter3)

In [64]:
qp = QueryParser()
qp.initFile('../easyCLEF08/easyCLEF08_query.txt', '../easyCLEF08/easyCLEF08_gt.txt')
qu = qp.nextQuery()
print(qu.text_, '\n', qu.relevants_)
prec = PrecisionAtN()
prec_cluster = ClusterRecallAtN()
rank = irm2.getRanking(qu.text_)

irlist = IRList(qu, rank)
print(prec.eval(ir_list=irlist))
print(prec_cluster.eval(irlist))

       church with more than two towers. Relevant images will show a church, cathedral or a mosque with three or more towers. Churches with only one or two towers are not relevant. Buildings that are not churches, cathedrals or mosques are not relevant even if they have more than two towers. 
 {37353: 1, 37368: 1, 37371: 1, 37374: 1, 37375: 1, 37376: 1, 37386: 1, 37387: 1, 37392: 1, 37369: 1, 37393: 1, 37394: 1, 37169: 2, 37194: 2, 37204: 2, 37191: 2, 37192: 2, 31337: 3, 40500: 3, 32763: 4, 32866: 4, 32870: 4, 39625: 4, 37303: 5}
0.047619047619047616
0.2


In [65]:
from diversity.Clustering import ClusteringDiversifier

In [69]:
rank

array([[35878, 0.1439165525922309],
       [35880, 0.13934660285832354],
       [36043, 0.13934660285832354],
       ...,
       [40667, 0],
       [40675, 0],
       [40688, 0]], dtype=object)

In [113]:
cd = ClusteringDiversifier(index, cluster_order_by='size_descending')

data, cluster_order, div = cd.diversify(irlist, by_top_n=30, n_clusters=6)
print(cluster_order)
print(div)
data

1    1
2    2
5    5
0    0
3    3
4    4
Name: cluster, dtype: int64
[15763, 21420, 35878, 4898, 35621, 36043, 23959, 32866, 35880, 6246, 15179, 6636, 10651, 37105, 37069, 4945, 36072, 8539, 22874, 36023, 37070, 11512, 36071, 37072, 8054, 37106, 37158, 37101, 37159, 37068]


,id,rank,cluster
14,15763,15,0
11,21420,12,1
0,35878,1,2
6,4898,7,3
8,35621,9,4
2,36043,3,5
25,23959,26,0
12,32866,13,1
1,35880,2,2
7,6246,8,3


In [109]:
data.index

Int64Index([ 4, 21, 10,  7, 23, 13, 15, 18, 16,  3,  0, 20,  9,  6,  5,  2,  8,
            17,  1, 12, 25, 29, 14, 19, 24, 27, 11, 22, 26, 28],
           dtype='int64')

In [85]:
data[1]

1

In [76]:
data.sort_values(by='min_rank')['cluster']


1    1
4    4
5    5
0    0
2    2
3    3
Name: cluster, dtype: int64

In [107]:
5 % 5

0